In [1]:
using CSV
using Distances
using Random

In [2]:
Random.seed!(1)
n = 20      # The number of patients
c_2 = 5     # unit distance cost
patients = [2:n+1;]
V = [1:n+1;]  # 1 denotes depot.    
q = [rand(12:15,n+1);]   # Requirment of each patient
arcs = []
for i in V
    for j in V 
        push!(arcs,(i,j))
    end
end
Q = 100    # Capacity
D = 10  # Max flying distance
q

21-element Array{Int64,1}:
 14
 14
 15
 13
 13
 14
 14
 15
 14
 15
 12
 12
 12
 14
 12
 13
 14
 15
 15
 13
 14

In [3]:
data = CSV.read("Patients21239_TableToExcel.csv")

,OID,CID,POINT_X,POINT_Y,Column5,Column6,Column7,Column8,Column9,Column10
,Int64,Int64,Float64,Float64,Missing,Missing,Missing,Missing,Missing,Missing
1,0,0,-76.5831,39.3669,missing,missing,missing,missing,missing,missing
2,1,1,-76.5872,39.3697,missing,missing,missing,missing,missing,missing
3,2,2,-76.5965,39.3464,missing,missing,missing,missing,missing,missing
4,3,3,-76.5986,39.3515,missing,missing,missing,missing,missing,missing
5,4,4,-76.5945,39.3565,missing,missing,missing,missing,missing,missing
6,5,5,-76.5899,39.3488,missing,missing,missing,missing,missing,missing
7,6,6,-76.5735,39.3576,missing,missing,missing,missing,missing,missing
8,7,7,-76.595,39.3667,missing,missing,missing,missing,missing,missing
9,8,8,-76.579,39.3662,missing,missing,missing,missing,missing,missing


In [4]:
x_loc = data[:,3];
y_loc = data[:,4];
d = zeros(n+1,n+1)   # Distance matrix
for i = 1: length(V)
    for j = 1: length(V)
        d[i,j] = haversine((x_loc[i],y_loc[i]),(x_loc[j],y_loc[j]),6372.8) 
    end
end
d

21×21 Array{Float64,2}:
 0.0       0.473235  2.55293   2.16895   …  0.62688   1.39412   0.775418
 0.473235  0.0       2.7143    2.25302      1.0621    1.73071   0.580527
 2.55293   2.7143    0.0       0.588874     2.15083   1.39651   3.26439 
 2.16895   2.25302   0.588874  0.0          1.88448   1.29525   2.82342 
 1.51647   1.60119   1.13377   0.657355     1.29679   0.936856  2.16795 
 2.096     2.34031   0.627563  0.800846  …  1.61317   0.803975  2.84938 
 1.31568   1.78417   2.33752   2.25684      0.764532  0.966627  1.98035 
 1.02297   0.750379  2.26006   1.72105      1.34235   1.66756   1.29829 
 0.356309  0.803189  2.66309   2.3426       0.543224  1.38018   0.920459
 0.33319   0.713831  2.23966   1.88607      0.357911  1.06276   1.10629 
 1.18923   1.3455    1.37674   0.988424  …  0.928715  0.698526  1.88786 
 1.91489   1.46443   3.8928    3.34159      2.52566   3.15455   1.33076 
 1.81938   2.13342   1.08015   1.15229      1.26789   0.431559  2.59231 
 0.741591  0.904095  3.2644

In [5]:
using JuMP, Gurobi 
v = 1:5  # # of drones
m = Model(Gurobi.Optimizer);
@variable(m,x[arcs,v],Bin) #arcs covered by vehicle k
@variable(m,u[1:n+1,v])
# Each patient will be visited exactly once
@constraint(m,single1[j in patients],sum(x[(i,j),k] for i in V for k in v)==1) 
@constraint(m,single2[i in patients],sum(x[(i,j),k] for j in V for k in v)==1) 
# Drone starts from depot
@constraint(m,depot1[k in v],sum(x[(1,j),k] for j in patients)==1)
# Drone ends at depot
@constraint(m,depot2[k in v],sum(x[(i,1),k] for i in patients)==1)
# Capacity constriant
@constraint(m,capacity[k in v],sum(q[i]*x[(i,j),k] for i in patients for j in V)<=Q)
# Balance constraint
@constraint(m,balance[h in patients,k in v],sum(x[(i,h),k] for i in V)-sum(x[(h,j),k] for j in V)==0) 
# Subtour Elimination MTZ
@constraint(m,subtour1[k in v], u[1,k]==1)
@constraint(m,subtour2[k in v,i in 2:n+1], u[i,k]>=2)   
@constraint(m,subtour3[k in v,i in 2:n+1], uwwwwi,k]<=n)
@constraint(m,subtour4[k in v,i in 2:n+1, j in 2:n+1], (u[i,k]-u[j,k]+1)<=(n-1)*(1-x[(i,j),k]))
# Maximum flying distance for each drone
@constraint(m,maxdis[k in v], sum(d[i,j]*x[(i,j),k] for i in V for j in V)<=D)                                             
# Objective Function
@objective(m,Min,5*sum(d[i,j]*x[(i,j),k] for i in V for j in V for k in v))                                                              
optimize!(m)
@show objective_value(m);

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 2365 rows, 2310 columns and 18605 nonzeros
Model fingerprint: 0x9d572e9a
Variable types: 105 continuous, 2205 integer (2205 binary)
Coefficient statistics:
  Matrix range     [2e-01, 2e+01]
  Objective range  [1e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+02]
Presolve removed 305 rows and 110 columns
Presolve time: 0.03s
Presolved: 2060 rows, 2200 columns, 17900 nonzeros
Variable types: 100 continuous, 2100 integer (2100 binary)

Root relaxation: objective 6.841413e+01, 112 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   68.41413    0   30          -   68.41413      -     -    0s
H    0     0                     170.9205050   68.41413  60.0%    

In [6]:
val = value.(x)
for k in v
    for i in V
        for j in V
            if val[(i,j),k]==1
                println(x[(i,j),k])
            end 
        end
    end
end

x[(1, 13),1]
x[(3, 4),1]
x[(4, 5),1]
x[(5, 11),1]
x[(6, 15),1]
x[(11, 1),1]
x[(13, 6),1]
x[(15, 3),1]
x[(1, 8),2]
x[(8, 17),2]
x[(12, 21),2]
x[(14, 18),2]
x[(16, 14),2]
x[(17, 12),2]
x[(18, 1),2]
x[(21, 16),2]
x[(1, 9),3]
x[(9, 1),3]
x[(1, 2),4]
x[(2, 1),4]
x[(1, 19),5]
x[(7, 20),5]
x[(10, 1),5]
x[(19, 7),5]
x[(20, 10),5]
